In [6]:
import cv2
import numpy as np
import os
import matplotlib
%matplotlib inline

import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks.callbacks import ModelCheckpoint
from keras.layers.pooling import AveragePooling2D, MaxPooling2D
import tensorflow as tf
from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers import Input, Convolution2D, BatchNormalization
from keras.models import Model, Sequential
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report 
from imutils import paths
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import pandas as pd
import argparse
import pickle
import cv2
import os   
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix
import itertools
from sklearn.metrics import accuracy_score, recall_score, precision_score
from keras.models import load_model

Using TensorFlow backend.


In [11]:
def load_FireNetMobile():
    model = Sequential()

    data_input_shape = (64,64,3)

    model.add(Convolution2D(64, (3,3),padding='same',activation='relu', input_shape=data_input_shape))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.1))

    model.add(Convolution2D(32, (5,5),padding='same',activation='relu'))
    model.add(BatchNormalization())
    model.add(AveragePooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.1))

    model.add(Convolution2D(32, (3,3),padding='same',activation='relu'))
    model.add(BatchNormalization())
    model.add(AveragePooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.1))

    model.add(Flatten())
    model.add(Dense(64, name='low_output'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    weights_path = 'Trained models/FireNetMobile_large.h5'
    model.load_weights(weights_path)
    
    intermediate_layer_model = Model(inputs=model.input,
                                 outputs=model.get_layer('low_output').output)
    
    return intermediate_layer_model


def load_FireNet():
    model = Sequential()

    data_input_shape = (224,224,3)

    model.add(Convolution2D(128, (3,3),padding='same',activation='relu', input_shape=data_input_shape))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.1))

    model.add(Convolution2D(64, (3,3),padding='same',activation='relu'))
    model.add(BatchNormalization())
    model.add(AveragePooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Convolution2D(128, (3,3),padding='same',activation='relu'))
    model.add(BatchNormalization())
    model.add(AveragePooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Convolution2D(64, (3,3),padding='same',activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.1))

    model.add(Flatten())
    model.add(Dense(64, name='high_output'))
    model.add(Dropout(0.5))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation='sigmoid'))

    weights_path = 'Trained models/FireNet_large_new.h5'
    model.load_weights(weights_path)
    
    intermediate_layer_model = Model(inputs=model.input,
                                 outputs=model.get_layer('high_output').output)
    
    return intermediate_layer_model

In [12]:
model_low = load_FireNetMobile()
model_high = load_FireNet()

In [59]:
path_labels = '../Datasets/Videos/Rescuer_61/labels/'
path_videos = '../Datasets/Videos/Rescuer_61/'
path_proc = '../Datasets/Videos/Rescuer_61/proc/'

In [61]:
def proc(path):
    labels = pd.read_csv(path_labels + path[:path.rfind('.')] + '.csv', header=None)[0]
    #labels[0] = labels[0].apply(lambda x: x if x == 'fire' else 'normal')
    #dct = dict(labels)
    
    new_labels = []
    features_low_grid = [[] for _ in range(9)]
    features_high = []
    
    vs = cv2.VideoCapture(path_videos + path)
    
    cnt = 0
    total = 0
    
    while True:
        
        if cnt % 10 != 0:
            cnt += 1
            grabbed, frame = vs.read()
            if not grabbed:
                break
            continue
        
        grabbed, frame = vs.read()
        
        if not grabbed:
            break
            
        total += 1
        
        if total == 100:
            break
            
        new_labels.append(labels.iloc[cnt])
        cnt += 1
            
        frame = cv2.resize(frame, (224, 224))
    
        features_high.append(model_high.predict(np.expand_dims(frame, axis=0))[0])

        y = 0
        pos = 0
        while y < 224:
            x = 0
            while x < 224:
                features_low_grid[pos].append(model_low.predict(np.expand_dims( \
                    frame[x: x+64, y: y+64], axis=0))[0])
                x += 80
                pos += 1
            y += 80

    dct = {'label': new_labels}
    main_crop_names = ['main_crop_{}'.format(i) for i in range(64)]
    dct.update(dict(zip(main_crop_names, np.array(features_high).T)))
    grid_crop_names = ['grid_crop_{}'.format(i) for i in range(64 * 9)]
    dct.update(dict(zip(grid_crop_names, np.hstack(features_low_grid).T)))

    df = pd.DataFrame(dct)
    df.to_csv(path_proc + path[:path.rfind('.')] + '.csv')

In [ ]:
for file in os.listdir(path_videos):
    if file.endswith(".mp4") or file.endswith(".avi"):
        proc(file)
    print('ok')

In [67]:
image = cv2.imread('background.jpeg')
image = cv2.resize(image, (224, 224))

In [70]:
high = model_high.predict(np.expand_dims(image, axis=0))[0]

low = [None] * 9

y = 0
pos = 0
while y < 224:
    x = 0
    while x < 224:
        low[pos] = model_low.predict(np.expand_dims( \
            image[x: x+64, y: y+64], axis=0))[0]
        x += 80
        pos += 1
    y += 80

In [73]:
low

[array([-360.59433  ,  439.84277  ,  308.7743   ,  112.67467  ,
         285.78165  ,  301.2428   ,  131.70412  ,  553.47253  ,
        -422.1458   ,  114.67339  ,  119.02037  ,  118.2842   ,
         305.09735  ,   45.610085 ,  498.92282  ,    0.9682052,
        -767.481    ,  443.4135   , -542.2606   , -230.38405  ,
        -454.       , -219.47952  , -760.9215   ,   32.50014  ,
        -499.54718  , -713.5107   ,   35.739532 , -408.09442  ,
        -475.0423   ,   44.642574 ,   24.946404 , -509.61832  ,
         666.6748   ,   15.535296 , -591.8725   , -208.14088  ,
          97.16079  , -698.0067   ,  304.70676  , -674.5176   ,
         527.4608   ,  309.17142  ,  277.84387  ,  224.83998  ,
         429.30786  ,  211.21483  ,  334.22513  , -653.1048   ,
        -896.7906   , -259.32785  ,  103.48495  ,  413.75558  ,
         -35.50561  , -487.76862  ,  251.40257  , -540.0857   ,
         155.2368   ,  168.00641  ,  281.8337   , -723.6079   ,
        -661.8852   ,  -49.251995 ,  336

In [76]:
high

array([  -477.52917, -11071.454  ,  -5949.0967 ,  -1316.1088 ,
         -619.5017 ,    315.5363 ,  -3065.6626 , -13078.517  ,
        -9436.598  ,  -7485.3696 ,  -2816.1313 ,   5441.57   ,
        -2376.2205 ,  -3693.2568 ,  -4329.305  , -11984.808  ,
        -3086.3193 ,  -3739.7612 ,  -2878.1106 ,  -1728.7504 ,
        -2307.7732 ,  -6855.501  ,   3293.864  ,  -8407.504  ,
        -3006.8816 ,  -9363.95   , -12925.878  ,  -8094.27   ,
         -351.56946,  -6414.8306 ,  -3635.5535 ,  -1558.3734 ,
        -6911.316  ,  -5718.05   , -12836.288  ,   1055.9739 ,
        -3802.5674 ,  -4323.298  ,  -1467.364  ,  -3217.269  ,
        -2590.2834 ,  -3242.7852 ,  -3349.867  ,  -2172.4604 ,
        -3004.6567 ,  -6095.2197 ,  -6279.2944 ,  -7141.881  ,
        -1502.9889 ,  -3438.187  ,  -9514.903  ,   -190.90501,
        -2599.5923 ,  -7268.642  ,    107.29111,   5675.97   ,
        -3733.056  ,  -4252.6475 ,  -2776.336  ,  -4904.822  ,
        -8774.311  ,  -9599.523  ,  -5652.8164 ,   -944

In [ ]:
dct = {'label': new_labels}
main_crop_names = ['main_crop_{}'.format(i) for i in range(64)]
dct.update(dict(zip(main_crop_names, np.array(features_high).T)))
grid_crop_names = ['grid_crop_{}'.format(i) for i in range(64 * 9)]
dct.update(dict(zip(grid_crop_names, np.hstack(features_low_grid).T)))

df = pd.DataFrame(dct)
df.to_csv(path_proc + path[:path.rfind('.')] + '.csv')

In [ ]:
arr = [np.array([11, 12]), np.array([21, 22])]


In [78]:
list(np.array([21, 22]))

[21, 22]